In [3]:
import pandas as pd
import json
import numpy as np
from tqdm.notebook import tqdm
import swifter
import dask.dataframe as dd
from dask.multiprocessing import get
from pandarallel import pandarallel
import time
from collections import Counter,defaultdict
import csv
import glob
import gzip
import json
import numpy as np
import os
import pandas as pd
from collections import Counter
from collections import defaultdict
import gzip
from tqdm.notebook import tqdm
tqdm.pandas()
pandarallel.initialize()

/home/xh3993/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


##### Biology processed

In [12]:
Biology=pd.read_csv("/data/openalex/processed/disciplines/Biology.csv.gz", compression='gzip', quotechar='"')

In [13]:
print('original biology publication number', len(Biology)

30046001

In [15]:
with open('/data/openalex/processed/work_author.json') as f:
    work_author=json.load(f)

In [16]:
Biology['author']=Biology['work_id'].apply(lambda x: work_author[x] if x in work_author else np.nan)

In [22]:
with open("/data/openalex/processed/reference_biology.json") as f:
    work_reference=json.load(f)

In [35]:
# references within the field
Biology['reference']=Biology['work_id'].apply(lambda x: work_reference[x] if x in work_reference else np.nan)

In [26]:
Biology.head(1)

,work_id,year,author,reference
0,W3173853689,2020.0,"[A3173361891, A3175006382, A2986250374, A30870...","[W2104494153, W2103950684, W2766885277, W25085..."


In [28]:
Biology['len']=Biology['author'].apply(lambda x: len(x) if x==x else 0)

In [29]:
W_time={}
for i,j in zip(Biology['work_id'],Biology['year']):
    W_time[i]=j

In [40]:
W_pre2=defaultdict(lambda:[])
W_pre5=defaultdict(lambda:[])
W_pre10=defaultdict(lambda:[])
for i,j in zip(Biology['work_id'],Biology['reference']):
    if j==j:
        for m in j:
            if m in W_time:
                diff=W_time[i]-W_time[m]
                if 0<=diff<=2:
                    W_pre2[m].append(i)
                if 0<=diff<=5:
                    W_pre5[m].append(i)
                if 0<=diff<=10:
                    W_pre10[m].append(i)

In [41]:
Biology['citation2']=Biology['work_id'].apply(lambda x: len(W_pre2[x]))

In [42]:
Biology['citation5']=Biology['work_id'].apply(lambda x: len(W_pre5[x]))

In [44]:
Biology['citation10']=Biology['work_id'].apply(lambda x: len(W_pre10[x]))

In [47]:
print(Biology['citation2'].sum()/Biology['citation5'].sum())
print(Biology['citation5'].sum()/Biology['citation10'].sum())

0.44271792424262885
0.6281436640881887


In [48]:
Biology=Biology.dropna(subset=['year','author'])

In [52]:
Biology.head()

,work_id,year,author,reference,len,citation2,citation5,citation10
0,W3173853689,2020.0,"[A3173361891, A3175006382, A2986250374, A30870...","[W2104494153, W2103950684, W2766885277, W25085...",5,0,0,0
1,W4280522669,2022.0,"[A4281201143, A4281201144]",NaN,2,0,0,0
2,W2092834612,2013.0,"[A2516971166, A2553500975, A3172661203, A24931...","[W1511273783, W1535380492, W1966065368, W19725...",5,11,23,40
3,W2134880685,2015.0,"[A2104997395, A2502458046, A2305943237, A24012...","[W1573966200, W1729331993, W1744995852, W19692...",9,6,16,23
4,W2401666164,2016.0,"[A2250607093, A2136793144, A2309608514]","[W635674086, W1607454482, W1964581329, W196647...",3,9,26,30


In [53]:
year=[[i]*int(j) for i,j in zip(Biology['year'].values,Biology['len'].values)]

In [54]:
year=[j for i in year for j in i]
aid=[j for i in Biology['author'].values for j in i]

In [55]:
W=defaultdict(lambda:[])
for i,j in zip(aid,year):
    if j==j:
        W[i].append(j)

In [56]:
def sci_age(x):
    try:
        return [x['year']-np.min(W[i])+1 if i!='na' else np.nan for i in x['author']]
    except:
        return np.nan

In [57]:
Biology['sci_age']=Biology.progress_apply(lambda x: sci_age(x), axis=1)

  0%|          | 0/29294509 [00:00<?, ?it/s]

In [58]:
#select year from 1800-2023
Biology=Biology[(Biology['year']<=2023) & (Biology['year']>=1800)]

In [61]:
#remove papers with authors whose career age is above 80
Biology['remove80']=Biology['sci_age'].progress_apply(lambda x: len([i for i in x if i>80]) if x==x else x)

  0%|          | 0/29293434 [00:00<?, ?it/s]

In [62]:
Biology=Biology[Biology['remove80']==0]

In [63]:
print('removing above 80 and outliner years',len(Biology))

removing above 80 and outliner years 29239800


In [64]:
#teams with at least two people
Biology=Biology[Biology['len']>=2]

In [65]:
print('team publications',len(Biology))

team publications 22678629


In [66]:
Biology.columns

Index(['work_id', 'year', 'author', 'reference', 'len', 'citation2',
       'citation5', 'citation10', 'sci_age', 'remove80'],
      dtype='object')

In [67]:
Biology=Biology[['work_id', 'author', 'year','reference', 'len',
       'citation2', 'citation5', 'citation10', 'sci_age']]

In [68]:
Biology.to_csv('/data/openalex/processed/sharedleadership/Biology_teams.csv',index=False)

##### Medicine processed

In [2]:
def processed (input_file, output_file,reference_file): #input #output reference, author, sci_age, c2, c5,c10
    Biology=pd.read_csv(input_file, compression='gzip', quotechar='"')
    print('original publication number', len(Biology))
    with open('/data/openalex/processed/work_author.json') as f:
        work_author=json.load(f)
    Biology['author']=Biology['work_id'].apply(lambda x: work_author[x] if x in work_author else np.nan)
    with open(reference_file) as f:
        work_reference=json.load(f)
    # references within the field
    Biology['reference']=Biology['work_id'].apply(lambda x: work_reference[x] if x in work_reference else np.nan)
    Biology['len']=Biology['author'].apply(lambda x: len(x) if x==x else 0)
    W_time={}
    for i,j in zip(Biology['work_id'],Biology['year']):
        W_time[i]=j
    W_pre2=defaultdict(lambda:[])
    W_pre5=defaultdict(lambda:[])
    W_pre10=defaultdict(lambda:[])
    for i,j in zip(Biology['work_id'],Biology['reference']):
        if j==j:
            for m in j:
                if m in W_time:
                    diff=W_time[i]-W_time[m]
                    if 0<=diff<=2:
                        W_pre2[m].append(i)
                    if 0<=diff<=5:
                        W_pre5[m].append(i)
                    if 0<=diff<=10:
                        W_pre10[m].append(i)
    Biology['citation2']=Biology['work_id'].apply(lambda x: len(W_pre2[x]))
    Biology['citation5']=Biology['work_id'].apply(lambda x: len(W_pre5[x]))
    Biology['citation10']=Biology['work_id'].apply(lambda x: len(W_pre10[x]))
    print(Biology['citation2'].sum()/Biology['citation5'].sum())
    print(Biology['citation5'].sum()/Biology['citation10'].sum())  
    Biology=Biology.dropna(subset=['year','author'])
    year=[[i]*int(j) for i,j in zip(Biology['year'].values,Biology['len'].values)]
    year=[j for i in year for j in i]
    aid=[j for i in Biology['author'].values for j in i]
    W=defaultdict(lambda:[])
    for i,j in zip(aid,year):
        if j==j:
            W[i].append(j)
    def sci_age(x):
        try:
            return [x['year']-np.min(W[i])+1 if i!='na' else np.nan for i in x['author']]
        except:
            return np.nan
    Biology['sci_age']=Biology.progress_apply(lambda x: sci_age(x), axis=1)
    #select year from 1800-2023
    Biology=Biology[(Biology['year']<=2023) & (Biology['year']>=1800)]
    #remove papers with authors whose career age is above 80
    Biology['remove80']=Biology['sci_age'].progress_apply(lambda x: len([i for i in x if i>80]))
    Biology=Biology[Biology['remove80']==0]
    print('removing above 80 and outliner years',len(Biology))
    #teams with at least two people
    Biology=Biology[Biology['len']>=2]     
    print('team publications',len(Biology))
    Biology=Biology[['work_id', 'author', 'year', 'reference', 'len',
       'citation2', 'citation5', 'citation10', 'sci_age']]
    Biology.to_csv(output_file,index=False)

In [3]:
processed("/data/openalex/processed/disciplines/Medicine.csv.gz",'/data/openalex/processed/sharedleadership/Medicine_teams.csv',"/data/openalex/processed/reference_biology.json")

original publication number 44335837
0.4487259975370613
0.6408345450025145


  0%|          | 0/41525790 [00:00<?, ?it/s]

  0%|          | 0/41524731 [00:00<?, ?it/s]

removing above 80 and outliner years 41280524
team publications 30212086


In [4]:
CS=pd.read_csv("/data/openalex/processed/disciplines/Computer science.csv.gz", compression='gzip', quotechar='"')

In [8]:
W_CS={}
for i in CS['work_id'].values:
    W_CS[i]='CS'

reference_CS=defaultdict(lambda:[])
with gzip.open("/data/openalex/csv-files/works_referenced_works.csv.gz",'rt') as f:
    for e,line in tqdm(enumerate(f)):
        if e>0:
            i=line.split(',')[0].split('/')[-1]
            j=line.split(',')[1].split('/')[-1].split('\n')[0]
            try:
                if W_CS[i]==W_CS[j]:
                    reference_CS[i].append(j)
            except:
                pass

with open("/data/openalex/processed/reference_CS.json",'w') as f:
    json.dump(reference_CS, f)

0it [00:00, ?it/s]

In [3]:
processed("/data/openalex/processed/disciplines/Computer science.csv.gz",'/data/openalex/processed/sharedleadership/Computer Science_teams.csv',"/data/openalex/processed/reference_CS.json")

original publication number 33674530
0.4677328564021705
0.6614499295402404


  0%|          | 0/30717070 [00:00<?, ?it/s]

  0%|          | 0/30716051 [00:00<?, ?it/s]

removing above 80 and outliner years 30677568
team publications 19902081


In [2]:
def processed (input_file, output_file,reference_file): #input #output reference, author, sci_age, c2, c5,c10
    CS=pd.read_csv(input_file, compression='gzip', quotechar='"')
    W_CS={}
    for i in CS['work_id'].values:
        W_CS[i]='CS'

    reference_CS=defaultdict(lambda:[])
    with gzip.open("/data/openalex/csv-files/works_referenced_works.csv.gz",'rt') as f:
        for e,line in tqdm(enumerate(f)):
            if e>0:
                i=line.split(',')[0].split('/')[-1]
                j=line.split(',')[1].split('/')[-1].split('\n')[0]
                try:
                    if W_CS[i]==W_CS[j]:
                        reference_CS[i].append(j)
                except:
                    pass

    with open(reference_file,'w') as f:
        json.dump(reference_CS, f) 

    #clear storage
    CS=0
    W_CS={}
    reference_CS=defaultdict(lambda:[])
    
    Biology=pd.read_csv(input_file, compression='gzip', quotechar='"')
    print('original publication number', len(Biology))
    with open('/data/openalex/processed/work_author.json') as f:
        work_author=json.load(f)
    Biology['author']=Biology['work_id'].apply(lambda x: work_author[x] if x in work_author else np.nan)
    with open(reference_file) as f:
        work_reference=json.load(f)
    # references within the field
    Biology['reference']=Biology['work_id'].apply(lambda x: work_reference[x] if x in work_reference else np.nan)
    Biology['len']=Biology['author'].apply(lambda x: len(x) if x==x else 0)
    W_time={}
    for i,j in zip(Biology['work_id'],Biology['year']):
        W_time[i]=j
    W_pre2=defaultdict(lambda:[])
    W_pre5=defaultdict(lambda:[])
    W_pre10=defaultdict(lambda:[])
    for i,j in zip(Biology['work_id'],Biology['reference']):
        if j==j:
            for m in j:
                if m in W_time:
                    diff=W_time[i]-W_time[m]
                    if 0<=diff<=2:
                        W_pre2[m].append(i)
                    if 0<=diff<=5:
                        W_pre5[m].append(i)
                    if 0<=diff<=10:
                        W_pre10[m].append(i)
    Biology['citation2']=Biology['work_id'].apply(lambda x: len(W_pre2[x]))
    Biology['citation5']=Biology['work_id'].apply(lambda x: len(W_pre5[x]))
    Biology['citation10']=Biology['work_id'].apply(lambda x: len(W_pre10[x]))
    print(Biology['citation2'].sum()/Biology['citation5'].sum())
    print(Biology['citation5'].sum()/Biology['citation10'].sum())  
    Biology=Biology.dropna(subset=['year','author'])
    year=[[i]*int(j) for i,j in zip(Biology['year'].values,Biology['len'].values)]
    year=[j for i in year for j in i]
    aid=[j for i in Biology['author'].values for j in i]
    W=defaultdict(lambda:[])
    for i,j in zip(aid,year):
        if j==j:
            W[i].append(j)
    def sci_age(x):
        try:
            return [x['year']-np.min(W[i])+1 if i!='na' else np.nan for i in x['author']]
        except:
            return np.nan
    Biology['sci_age']=Biology.progress_apply(lambda x: sci_age(x), axis=1)
    #select year from 1800-2023
    Biology=Biology[(Biology['year']<=2023) & (Biology['year']>=1800)]
    #remove papers with authors whose career age is above 80
    Biology['remove80']=Biology['sci_age'].progress_apply(lambda x: len([i for i in x if i>80]))
    Biology=Biology[Biology['remove80']==0]
    print('removing above 80 and outliner years',len(Biology))
    #teams with at least two people
    Biology=Biology[Biology['len']>=2]     
    print('team publications',len(Biology))
    Biology=Biology[['work_id', 'author', 'year', 'reference', 'len',
       'citation2', 'citation5', 'citation10', 'sci_age']]
    Biology.to_csv(output_file,index=False)

In [5]:
for i in [ 'Business' , 'Art', 'Economics',
          'Philosophy', 'Mathematics',
       'Environmental science', 'Chemistry',
       'Materials science', 'Political science',
       'History', 'Geology', 'Geography']:#'Psychology', 'Physics','Sociology', 'Engineering', 
    print(i)
    processed("/data/openalex/processed/disciplines/"+str(i)+".csv.gz","/data/openalex/processed/sharedleadership/"+str(i)+"_teams.csv","/data/openalex/processed/reference_"+str(i)+".json")

Business


0it [00:00, ?it/s]

KeyboardInterrupt: 

In [4]:
def processed (input_file, output_file,reference_file): #input #output reference, author, sci_age, keep all teams not only shared leadership 
    Biology=pd.read_csv(input_file, compression='gzip', quotechar='"')
    print('original publication number', len(Biology))
    with open('/data/openalex/processed/work_author.json') as f:
        work_author=json.load(f)
    Biology['author']=Biology['work_id'].apply(lambda x: work_author[x] if x in work_author else np.nan)
    with open(reference_file) as f:
        work_reference=json.load(f)
    # references within the field
    Biology['reference']=Biology['work_id'].apply(lambda x: work_reference[x] if x in work_reference else np.nan)
    Biology['len']=Biology['author'].apply(lambda x: len(x) if x==x else 0)  
    Biology=Biology.dropna(subset=['year','author'])
    year=[[i]*int(j) for i,j in zip(Biology['year'].values,Biology['len'].values)]
    year=[j for i in year for j in i]
    aid=[j for i in Biology['author'].values for j in i]
    W=defaultdict(lambda:[])
    for i,j in zip(aid,year):
        if j==j:
            W[i].append(j)
    def sci_age(x):
        try:
            return [x['year']-np.min(W[i])+1 if i!='na' else np.nan for i in x['author']]
        except:
            return np.nan
    Biology['sci_age']=Biology.progress_apply(lambda x: sci_age(x), axis=1)
    #select year from 1800-2023
    Biology=Biology[(Biology['year']<=2023) & (Biology['year']>=1800)]
    #remove papers with authors whose career age is above 80
    Biology['remove80']=Biology['sci_age'].progress_apply(lambda x: len([i for i in x if i>80]))
    Biology=Biology[Biology['remove80']==0]
    print('removing above 80 and outliner years',len(Biology))
    #teams with at least two people
#     Biology=Biology[Biology['len']>=2]     
    print('team publications',len(Biology))
    Biology=Biology[['work_id', 'author', 'year', 'reference', 'len', 'sci_age']]
    Biology.to_csv(output_file,index=False)

In [5]:
processed("/data/openalex/processed/disciplines/Computer science.csv.gz","/data/openalex/processed/sharedleadership/Computer science_allteams.csv","/data/openalex/processed/reference_CS.json")

original publication number 33674530


  0%|          | 0/30717070 [00:00<?, ?it/s]

  0%|          | 0/30716051 [00:00<?, ?it/s]

removing above 80 and outliner years 30677568
team publications 30677568
